In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ds = pd.read_csv("/kaggle/input/cassandra23-ps-2/dataset/train_data.csv")
Y = ds.actual_time
X = ds.drop('actual_time',axis=1)
X_test = pd.read_csv("/kaggle/input/cassandra23-ps-2/dataset/test_data.csv")

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
def extract_locations(df,s):
    for st in s:
        df[st + '_state'] = df[st].str.extract(r'\((.*?)\)')
        df[st + '_loc'] = df[st].str.split('_').str[0]
    df.drop(s,axis = 1,inplace = True)
    return df
X = extract_locations(X,[
    'source_name',
    'destination_name'
])
X_test = extract_locations(X_test,[
    'source_name',
    'destination_name'
])

X.head()

In [ ]:
# def use_state_features(df):
#     df['state_pair'] = df['source_name_state'] + '-' + df['destination_name_state']
#     df['same_state'] = (df['source_name_state']== df['destination_name_state'])
#     # Create dictionary of unique state pairs and their corresponding integer values
#     unique_pairs = sorted(df['state_pair'].unique())
#     state_map = {state_pair: i for i, state_pair in enumerate(unique_pairs)}

#     # Map state pairs to integer values
#     df['state_pair'] = df['state_pair'].map(state_map)
#     return df
# X = use_state_features(X)
# X_test = use_state_features(X_test)
# X.head()

In [ ]:
def add_date_features(df,col):
    df[col] = pd.to_datetime(df[col])
    df[col + '_month'] = df[col].dt.month
    df[col+ '_weekday' ] = df[col].dt.weekday
    df[col+ '_day' ] = df[col].dt.day
    df[col+'_hour'] = df[col].dt.hour
    
    return df
X = add_date_features(X,'trip_creation_time')
X_test = add_date_features(X_test,'trip_creation_time')
X = add_date_features(X,'cutoff_timestamp')
X_test = add_date_features(X_test,'cutoff_timestamp')
X.head()

In [ ]:
# def add_weekdays(df):
#     ## Add Office Hour Column
#     df['trip_creation_time'] = pd.to_datetime(df['trip_creation_time'])
#     df['tct_month'] = df['trip_creation_time'].dt.month
#     df['tct_weekday'] = df['trip_creation_time'].dt.weekday
#     df['tct_hour'] = df['trip_creation_time'].dt.hour
#     df['tct_day'] = df['trip_creation_time'].dt.day
    
#     return df
# X = add_weekdays(X)
# X_test = add_weekdays(X_test)

In [ ]:
def difference_od_time(df):
    # convert the "start" and "end" columns to datetime data types
    df['od_start_time'] = pd.to_datetime(df['od_start_time'])
    df['od_end_time'] = pd.to_datetime(df['od_end_time'])
    df['od_start_month'] = df['od_start_time'].dt.month
    df['od_start_weekday'] = df['od_start_time'].dt.weekday
    df['od_start_hour'] = df['od_start_time'].dt.hour
    df['od_start_day'] = df['od_start_time'].dt.day
    df['od_end_month'] = df['od_end_time'].dt.month
    df['od_end_weekday'] = df['od_end_time'].dt.weekday
    df['od_end_day'] = df['od_end_time'].dt.day
    df['od_end_hour'] = df['od_end_time'].dt.hour
    # calculate the duration between the "start" and "end" columns and store it in a new feature called "duration"
    df['duration'] = (df['od_end_time'] - df['od_start_time']).dt.total_seconds() / 3600
    return df
X = difference_od_time(X)
X_test = difference_od_time(X_test)
X.head()

In [ ]:
X['c_duration'] = np.abs(X['cutoff_timestamp'] - X['trip_creation_time']).dt.total_seconds() / 3600
X_test['c_duration'] = np.abs(X_test['cutoff_timestamp'] - X_test['trip_creation_time']).dt.total_seconds() / 3600

X['d_duration'] = np.abs(X['cutoff_timestamp'] - X['od_end_time']).dt.total_seconds() / 3600
X_test['d_duration'] = np.abs(X_test['cutoff_timestamp'] - X_test['od_end_time']).dt.total_seconds() / 3600

In [ ]:
X.head()

In [ ]:
for col in X.columns:
    if(X[col].dtype=='object'):
        print(f"No of Unique Values In {col} : {X[col].nunique()}")
print("\n")
drop_cols = [
    'trip_creation_time',
    'trip_uuid',
    'route_schedule_uuid',
    'od_start_time',
    'od_end_time',
    'cutoff_timestamp',
    'destination_center',
    'UID',
#     'source_center',
#     'state_pair',
#     'factor',
#     'cutoff_factor'
]
print(drop_cols)
X.drop(drop_cols,axis = 1,inplace = True)
X_test.drop(drop_cols,axis = 1,inplace = True)


for col in X.columns:
    if(X[col].dtype=='object'):
        print(f"No of Unique Values In {col} : {X[col].nunique()}")
print("\n")

In [ ]:
X.isna().sum()

In [ ]:
# # import the category_encoders library
# import category_encoders as ce

# def encode_cat_values(df, cols_to_encode,Y):
#     # loop over each column name in cols_to_encode and encode the column
#     for col in cols_to_encode:
#         # instantiate the target encoder
#         te = ce.TargetEncoder()
#         # fit and transform the training data
#         df[col] = te.fit_transform(df[col], Y)
#     return df

# cols_to_encode =  X.columns

# X = encode_cat_values(X, cols_to_encode,Y)
# X_test = encode_cat_values(X_test, cols_to_encode,Y)
# X.head()

In [ ]:
import numpy as np

# Check for missing or infinite values in the training data
# print(np.isnan(X[cols_with_missing]).sum())
# print(np.isinf(X[cols_with_missing]).sum())

# Check for missing or infinite values in the testing data
# print(np.isnan(X_test[cols_with_missing]).sum())
# print(np.isinf(X_test[cols_with_missing]).sum())

In [ ]:
# Replace missing values with the mean of the column
# X[cols_with_missing] = X[cols_with_missing].fillna(X[cols_with_missing].mean())
# X_test[cols_with_missing] = X_test[cols_with_missing].fillna(X_test[cols_with_missing].mean())

In [ ]:
from sklearn.preprocessing import LabelEncoder
def encode_cat_values(df,cols_to_encode):
    for col in cols_to_encode:
        le = LabelEncoder()
        le.fit(df[col])
        df[col] = le.transform(df[col]) 
    return df

cols_to_encode =  [
    'route_type',
    'source_name_state',
    'source_name_loc',
    'destination_name_state',
    'destination_name_loc',
    'is_cutoff',
#     'same_state',
    'source_center',
]

X = encode_cat_values(X,cols_to_encode)
X_test = encode_cat_values(X_test,cols_to_encode)
X.head()

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler
cols_with_missing = X.columns

# # Create a copy of the original data with only the selected columns
# train_cols = X[cols_with_missing].copy()
# test_cols = X_test[cols_with_missing].copy()

# # Create a KNN imputer object
# imputer = KNNImputer(n_neighbors=3)

# # Fit the imputer on the training data
# imputer.fit(train_cols)

# # Impute the missing values in the training and testing data
# X[cols_with_missing] = imputer.transform(train_cols)
# X_test[cols_with_missing] = imputer.transform(test_cols)

# Create a SimpleImputer object to handle remaining missing values
simple_imputer = SimpleImputer(strategy='median')

# Fit the imputer on the training data
simple_imputer.fit(X[cols_with_missing])

# Impute remaining missing values in the training and testing data
X[cols_with_missing] = simple_imputer.transform(X[cols_with_missing])
X_test[cols_with_missing] = simple_imputer.transform(X_test[cols_with_missing])

# Scale the data
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# X = pd.DataFrame(X, columns=cols_with_missing)
# X_test = pd.DataFrame(X_test, columns=cols_with_missing)
# # Save the imputed data to a new CSV file
# X.to_csv('train_data_imputed.csv', index=False)
# X_test.to_csv('test_data_imputed

In [ ]:
X.head()

In [ ]:
X.isna().sum()

In [ ]:
X_test.isna().sum()

In [ ]:
X['m2'] = X['segment_factor'] * X['segment_actual_time']
X_test['m2'] = X_test['segment_factor'] * X_test['segment_actual_time']

In [ ]:
X['m3'] = X['factor'] * X['actual_distance_to_destination']/X['start_scan_to_end_scan']
X_test['m3'] = X_test['factor'] * X_test['actual_distance_to_destination']/X_test['start_scan_to_end_scan']

In [ ]:
# X['m1'] = X['segment_factor'] * X['segment_osrm_distance']
# X_test['m1'] = X_test['segment_factor'] * X_test['segment_osrm_distance']

In [ ]:
##
X['cfXtme'] = X['cutoff_factor'] * X['start_scan_to_end_scan']
X_test['cfXtme'] = X_test['cutoff_factor'] * X_test['start_scan_to_end_scan']

In [ ]:
X['cfXdis'] = X['cutoff_factor'] * X['actual_distance_to_destination']
X_test['cfXdis'] = X_test['cutoff_factor'] * X_test['actual_distance_to_destination']

In [ ]:
# X['ratio'] = X['duration'] / X['start_scan_to_end_scan']
# X_test['ratio'] = X_test['duration'] / X_test['start_scan_to_end_scan']

In [ ]:
# X['ratio1'] = X['osrm_time'] / X['osrm_distance']
# X_test['ratio1'] = X_test['osrm_time'] / X_test['osrm_distance']

In [ ]:
# X['speed_factor'] = (X['segment_osrm_distance']/X['segment_osrm_time'])*X['segment_factor']
# X_test['speed_factor'] = (X_test['segment_osrm_distance']/X_test['segment_osrm_time'])*X_test['segment_factor']

In [ ]:
# X['dif'] = np.abs(X['duration']-X['start_scan_to_end_scan'])
# X_test['dif'] = np.abs(X_test['duration']-X_test['start_scan_to_end_scan'])

In [ ]:
X_test.isna().sum()

In [ ]:
X.isna().sum()

In [ ]:
# drop_cols = [
    
# ]
# print(drop_cols)
# X.drop(drop_cols,axis = 1,inplace = True)
# X_test.drop(drop_cols,axis = 1,inplace = True)


# for col in X.columns:
#     if(X[col].dtype=='object'):
#         print(f"No of Unique Values In {col} : {X[col].nunique()}")
# print("\n")

In [ ]:
# Necessary imports
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
X.head()

In [ ]:
X_test.head()

In [ ]:
X.isna().sum()

In [ ]:
X_test.isna().sum()

In [ ]:
# # Calculate IQR and Z-score
# Q1 = X.quantile(0.25)
# Q3 = X.quantile(0.75)
# IQR = Q3 - Q1
# Z = np.abs((X - X.mean()) / X.std())

# # Drop rows based on IQR or Z-score
# threshold = 3  # Change the threshold as needed
# X = X[(Z < threshold).all(axis=1)]
# Y = Y[(Z < threshold).all(axis=1)]

In [ ]:
# Create a SimpleImputer object to handle remaining missing values
simple_imputer = SimpleImputer(strategy='median')
cols_with_missing = X.columns
# Fit the imputer on the training data
simple_imputer.fit(X[cols_with_missing])

# Impute remaining missing values in the training and testing data
X[cols_with_missing] = simple_imputer.transform(X[cols_with_missing])
X_test[cols_with_missing] = simple_imputer.transform(X_test[cols_with_missing])

In [ ]:
# Necessary imports
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.feature_selection import SelectKBest, f_regression
import matplotlib.pyplot as plt

# Select the best 25 features using SelectKBest
selector = SelectKBest(f_regression, k=30)
X_new = selector.fit_transform(X, Y)


# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_new, Y, test_size=0.2, random_state=42)

# Instantiation
model = XGBRegressor(n_estimators=2000, learning_rate=0.1, n_jobs=12, random_state=42, reg_lambda=0, eval_metric="rmse")

# Fit the model
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=100)

# Predict the model
pred = model.predict(X_val)

# RMSE Computation
rmse = np.sqrt(MSE(y_val, pred))
print("RMSE: %f" % rmse)
mae = MSE(y_val, pred)
print('Mean Absolute Error:', mae)

# Plot the training vs validation loss
results = model.evals_result()
epochs = len(results['validation_0']['rmse'])
x_axis = range(0, epochs)
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
ax.plot(x_axis, results['validation_1']['rmse'], label='Validation')
ax.legend()
plt.ylabel('RMSE')
plt.title('XGBoost RMSE')
plt.show()

In [ ]:
# [0]	validation_0-rmse:658.86391	validation_1-rmse:658.43555
# [100]	validation_0-rmse:21.40355	validation_1-rmse:31.44233
# [200]	validation_0-rmse:17.63907	validation_1-rmse:29.37174
# [300]	validation_0-rmse:15.31441	validation_1-rmse:28.28484
# [400]	validation_0-rmse:13.81312	validation_1-rmse:27.73792
# [500]	validation_0-rmse:12.65215	validation_1-rmse:27.41019
# [600]	validation_0-rmse:11.63584	validation_1-rmse:27.13704
# [700]	validation_0-rmse:10.87352	validation_1-rmse:26.90932
# [800]	validation_0-rmse:10.16310	validation_1-rmse:26.76307
# [900]	validation_0-rmse:9.59500	validation_1-rmse:26.66270
# [1000]	validation_0-rmse:9.01658	validation_1-rmse:26.55523
# [1100]	validation_0-rmse:8.43508	validation_1-rmse:26.41465
# [1200]	validation_0-rmse:8.02616	validation_1-rmse:26.37314
# [1300]	validation_0-rmse:7.58841	validation_1-rmse:26.29825
# [1400]	validation_0-rmse:7.26440	validation_1-rmse:26.21968
# [1500]	validation_0-rmse:6.96447	validation_1-rmse:26.17715
# [1600]	validation_0-rmse:6.61485	validation_1-rmse:26.13315
# [1700]	validation_0-rmse:6.36221	validation_1-rmse:26.11863
# [1800]	validation_0-rmse:6.10541	validation_1-rmse:26.10158
# [1900]	validation_0-rmse:5.85843	validation_1-rmse:26.06784
# [2000]	validation_0-rmse:5.65532	validation_1-rmse:26.03573
# [2100]	validation_0-rmse:5.40866	validation_1-rmse:26.01168
# [2200]	validation_0-rmse:5.20001	validation_1-rmse:25.99323
# [2300]	validation_0-rmse:4.99556	validation_1-rmse:25.97005
# [2400]	validation_0-rmse:4.83225	validation_1-rmse:25.96535
# [2500]	validation_0-rmse:4.65855	validation_1-rmse:25.95001
# [2600]	validation_0-rmse:4.50370	validation_1-rmse:25.94124
# [2700]	validation_0-rmse:4.35737	validation_1-rmse:25.93842
# [2800]	validation_0-rmse:4.20195	validation_1-rmse:25.92334
# [2900]	validation_0-rmse:4.06411	validation_1-rmse:25.91597
# [3000]	validation_0-rmse:3.91650	validation_1-rmse:25.90647
# [3100]	validation_0-rmse:3.77262	validation_1-rmse:25.90395
# [3200]	validation_0-rmse:3.64955	validation_1-rmse:25.90451
# [3300]	validation_0-rmse:3.53770	validation_1-rmse:25.89633
# [3400]	validation_0-rmse:3.40677	validation_1-rmse:25.88811
# [3500]	validation_0-rmse:3.30941	validation_1-rmse:25.88720
# [3600]	validation_0-rmse:3.18714	validation_1-rmse:25.87706
# [3700]	validation_0-rmse:3.07711	validation_1-rmse:25.87285
# [3800]	validation_0-rmse:2.98665	validation_1-rmse:25.86741
# [3900]	validation_0-rmse:2.91197	validation_1-rmse:25.86817
# [4000]	validation_0-rmse:2.83574	validation_1-rmse:25.86555
# [4100]	validation_0-rmse:2.75567	validation_1-rmse:25.86341
# [4200]	validation_0-rmse:2.68145	validation_1-rmse:25.86207
# [4300]	validation_0-rmse:2.59915	validation_1-rmse:25.86015
# [4400]	validation_0-rmse:2.52539	validation_1-rmse:25.85849
# [4500]	validation_0-rmse:2.45004	validation_1-rmse:25.85331
# [4600]	validation_0-rmse:2.37085	validation_1-rmse:25.84758
# [4700]	validation_0-rmse:2.30966	validation_1-rmse:25.84782
# [4800]	validation_0-rmse:2.24208	validation_1-rmse:25.84383
# [4900]	validation_0-rmse:2.17951	validation_1-rmse:25.84137
# [4999]	validation_0-rmse:2.11469	validation_1-rmse:25.83964
# RMSE: 25.839644
# Mean Absolute Error: 667.6871873158258

In [ ]:
importance = model.feature_importances_
# get feature names
feature_names = X.columns[selector.get_support()]
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %s, Score: %.5f' % (feature_names[i],v))
# plot feature importance
plt.bar(feature_names, importance)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# i = 0
# for col in X.columns:
#     print(f"{i} {col}")
#     i+=1

In [ ]:
from datetime import datetime
now = datetime.now().strftime("%m-%d_%H-%M")

Xt_new = selector.transform(X_test)
test_data = pd.read_csv('/kaggle/input/cassandra23-ps-2/dataset/test_data.csv')
y_test_pred = model.predict(Xt_new)

submission_df = pd.DataFrame({'UID': test_data['UID'], 'actual_time': y_test_pred})
submission_df.to_csv( 'At-'+now + '__RMSE-'+str(rmse)+'.csv', index=False)
print(f"Submission File Created At {now} with RMSE {rmse}")